In [1]:
import modulos as md
import numpy as np
import os
import sys
import constantes as cons
lista_filtrada = []
#lista=md.filtradodemonedas ()
lista = ['BTCUSDT', 'ETHUSDT', 'BCHUSDT', 'XRPUSDT', 'EOSUSDT', 'LTCUSDT', 'ETCUSDT', 'LINKUSDT', 'ADAUSDT', 'BNBUSDT', 'ATOMUSDT', 'KNCUSDT', 'DOGEUSDT', 'WAVESUSDT', 'MKRUSDT', 'DOTUSDT', 'CRVUSDT', 'TRBUSDT', 'RUNEUSDT', 'SOLUSDT', 'STORJUSDT', 'BLZUSDT', 'UNIUSDT', 'AVAXUSDT', 'FTMUSDT', 'TOMOUSDT', 'NEARUSDT', 'AAVEUSDT', 'FILUSDT', 'MATICUSDT', 'UNFIUSDT', 'LINAUSDT', '1000SHIBUSDT', 'MASKUSDT', 'DYDXUSDT', 'GALAUSDT', 'APEUSDT', 'OPUSDT', 'INJUSDT', 'LDOUSDT', 'APTUSDT', 'FETUSDT', 'MINAUSDT', 'CFXUSDT', 'STXUSDT', 'ARBUSDT', 'BLURUSDT', 'SUIUSDT', '1000PEPEUSDT', 'WLDUSDT', 'AGLDUSDT', 'YGGUSDT', 'CYBERUSDT', 'STRAXUSDT', 'LOOMUSDT', 'BIGTIMEUSDT', 'ORBSUSDT', 'POLYXUSDT']
#lista = ['BTCUSDT','LINKUSDT','BNBUSDT','DOGEUSDT','TRBUSDT','SOLUSDT','AVAXUSDT','1000SHIBUSDT','1000PEPEUSDT','WLDUSDT','BONDUSDT'] #ELEGIDOS
#lista = ['XRPUSDT']

if len(lista)>1:
    imprimo=False
    debug=False
else:
    imprimo= True
    debug=True
print(lista)
porcentajes_sumados=0
win_rate_buenos = 0
win_rate_malos = 0
trades = 0
for symbol in lista:    
    try:
        #print(symbol)
        data = md.estrategia_smart(symbol, debug = False, refinado = False, file_source = False, timeframe = '1h')
        resultado=md.backtesting_smart(data, plot_flag=imprimo, symbol=symbol)
        if resultado['Win Rate [%]'] >= 50:
            lista_filtrada.append(symbol)
        print(f"{symbol} - Return [%]: {md.truncate(resultado['Return [%]'],2)}% - # Trades: {resultado['# Trades']} - Profit Factor: {resultado['Profit Factor']} - Win Rate [%]: {resultado['Win Rate [%]']}")
        porcentajes_sumados=porcentajes_sumados+resultado['Return [%]']
        trades = trades + resultado['# Trades']
        if resultado['Win Rate [%]'] >= 50:
            win_rate_buenos = win_rate_buenos+1
        else:
            win_rate_malos = win_rate_malos+1
    except Exception as falla:
        _, _, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print("\nError: "+str(falla)+" - line: "+str(exc_tb.tb_lineno)+" - file: "+str(fname)+"\n")
        pass
for symbol in lista_filtrada:
    md.printandlog(cons.nombrelog,symbol,pal=1)
print(f"porcentajes_sumados {md.truncate(porcentajes_sumados,2)} - trades {trades} - win_rate_buenos {win_rate_buenos} - win_rate_malos {win_rate_malos}")
#(data.loc[data['Open Time'] == '2023-10-03 03:00:00'])


['BTCUSDT', 'ETHUSDT', 'BCHUSDT', 'XRPUSDT', 'EOSUSDT', 'LTCUSDT', 'ETCUSDT', 'LINKUSDT', 'ADAUSDT', 'BNBUSDT', 'ATOMUSDT', 'KNCUSDT', 'DOGEUSDT', 'WAVESUSDT', 'MKRUSDT', 'DOTUSDT', 'CRVUSDT', 'TRBUSDT', 'RUNEUSDT', 'SOLUSDT', 'STORJUSDT', 'BLZUSDT', 'UNIUSDT', 'AVAXUSDT', 'FTMUSDT', 'TOMOUSDT', 'NEARUSDT', 'AAVEUSDT', 'FILUSDT', 'MATICUSDT', 'UNFIUSDT', 'LINAUSDT', '1000SHIBUSDT', 'MASKUSDT', 'DYDXUSDT', 'GALAUSDT', 'APEUSDT', 'OPUSDT', 'INJUSDT', 'LDOUSDT', 'APTUSDT', 'FETUSDT', 'MINAUSDT', 'CFXUSDT', 'STXUSDT', 'ARBUSDT', 'BLURUSDT', 'SUIUSDT', '1000PEPEUSDT', 'WLDUSDT', 'AGLDUSDT', 'YGGUSDT', 'CYBERUSDT', 'STRAXUSDT', 'LOOMUSDT', 'BIGTIMEUSDT', 'ORBSUSDT', 'POLYXUSDT']


C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in double_scalars
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501


BTCUSDT - Return [%]: 7.53% - # Trades: 1 - Profit Factor: nan - Win Rate [%]: 100.0
ETHUSDT - Return [%]: -0.44% - # Trades: 1 - Profit Factor: 0.0 - Win Rate [%]: 0.0
BCHUSDT - Return [%]: 1.22% - # Trades: 4 - Profit Factor: 1.0816032945891985 - Win Rate [%]: 25.0
XRPUSDT - Return [%]: -6.21% - # Trades: 5 - Profit Factor: 0.0 - Win Rate [%]: 0.0
EOSUSDT - Return [%]: -3.33% - # Trades: 3 - Profit Factor: 0.0 - Win Rate [%]: 0.0
LTCUSDT - Return [%]: -2.36% - # Trades: 3 - Profit Factor: 0.0 - Win Rate [%]: 0.0
ETCUSDT - Return [%]: -0.79% - # Trades: 2 - Profit Factor: 0.13267408797355906 - Win Rate [%]: 50.0
LINKUSDT - Return [%]: 13.65% - # Trades: 3 - Profit Factor: 25.796158822937844 - Win Rate [%]: 66.66666666666666


C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in double_scalars
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501


ADAUSDT - Return [%]: 0.44% - # Trades: 1 - Profit Factor: nan - Win Rate [%]: 100.0
BNBUSDT - Return [%]: 2.66% - # Trades: 3 - Profit Factor: 3.695995203935893 - Win Rate [%]: 33.33333333333333
ATOMUSDT - Return [%]: -2.29% - # Trades: 3 - Profit Factor: 0.20353024522652946 - Win Rate [%]: 33.33333333333333
KNCUSDT - Return [%]: 9.25% - # Trades: 6 - Profit Factor: 3.8404727088211885 - Win Rate [%]: 33.33333333333333
DOGEUSDT - Return [%]: 11.18% - # Trades: 2 - Profit Factor: nan - Win Rate [%]: 100.0
WAVESUSDT - Return [%]: -0.98% - # Trades: 2 - Profit Factor: 0.0 - Win Rate [%]: 0.0
MKRUSDT - Return [%]: 3.98% - # Trades: 4 - Profit Factor: 2.459502707439588 - Win Rate [%]: 25.0
DOTUSDT - Return [%]: 2.23% - # Trades: 2 - Profit Factor: 2.9780250791974554 - Win Rate [%]: 50.0
CRVUSDT - Return [%]: -4.0% - # Trades: 3 - Profit Factor: 0.0 - Win Rate [%]: 0.0
TRBUSDT - Return [%]: 5.78% - # Trades: 2 - Profit Factor: 3.240767733587341 - Win Rate [%]: 50.0
RUNEUSDT - Return [%]: 9.6